# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as numpy
import pandas as pd 
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol  = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

0.5396554651367342

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [19]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    # print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ], index = my_columns), 
            ignore_index = True
        )

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [26]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

F:\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,72.31,4.34474,N/A
1,179,FCX,39.01,2.57932,N/A
2,441,TPR,45.24,2.54651,N/A
3,272,KSS,57.80,2.08873,N/A
4,253,IVZ,28.19,1.93749,N/A
5,148,DVN,30.08,1.91264,N/A
6,208,GPS,35.06,1.88408,N/A
7,305,MGM,44.89,1.8497,N/A
8,410,SIVB,590.75,1.80223,N/A
9,106,COF,165.87,1.70866,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [32]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number!')
        portfolio_size = input('Enter the size of your portfolio: ')
portfolio_input()

In [37]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
final_dataframe

F:\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,72.31,4.34474,2
1,179,FCX,39.01,2.57932,5
2,441,TPR,45.24,2.54651,4
3,272,KSS,57.80,2.08873,3
4,253,IVZ,28.19,1.93749,7
5,148,DVN,30.08,1.91264,6
6,208,GPS,35.06,1.88408,5
7,305,MGM,44.89,1.8497,4
8,410,SIVB,590.75,1.80223,0
9,106,COF,165.87,1.70866,1


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [89]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    # print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ], index = hqm_columns), ignore_index = True
        )

In [90]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.58,N/A,0.759551,N/A,0.262317,N/A,0.180868,N/A,0.105249,N/A,N/A
1,AAL,22.22,N/A,0.825478,N/A,0.432563,N/A,-0.032063,N/A,-0.072599,N/A,N/A
2,AAP,206.69,N/A,0.498012,N/A,0.283045,N/A,0.0994839,N/A,0.0904203,N/A,N/A
3,AAPL,136.61,N/A,0.531402,N/A,0.0120478,N/A,0.100143,N/A,0.0509849,N/A,N/A
4,ABBV,113.81,N/A,0.241434,N/A,0.121307,N/A,0.0824901,N/A,-0.0154749,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.09,N/A,0.423069,N/A,0.10152,N/A,0.0915425,N/A,-0.0228587,N/A,N/A
501,ZBH,170.32,N/A,0.46169,N/A,0.116641,N/A,0.0304813,N/A,0.0116628,N/A,N/A
502,ZBRA,520.12,N/A,1.11396,N/A,0.371481,N/A,0.0724165,N/A,0.0194799,N/A,N/A
503,ZION,58.20,N/A,0.807378,N/A,0.305002,N/A,-0.000860476,N/A,-0.0224571,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [91]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

hqm_dataframe = hqm_dataframe.dropna()

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(
            hqm_dataframe[change_col],
            hqm_dataframe.loc[row, change_col]) / 100

hqm_dataframe

F:\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.58,N/A,0.759551,0.742515,0.262317,0.660679,0.180868,0.884232,0.105249,0.938124,N/A
1,AAL,22.22,N/A,0.825478,0.780439,0.432563,0.9002,-0.032063,0.117764,-0.072599,0.0578842,N/A
2,AAP,206.69,N/A,0.498012,0.497006,0.283045,0.704591,0.0994839,0.654691,0.0904203,0.912176,N/A
3,AAPL,136.61,N/A,0.531402,0.532934,0.0120478,0.107784,0.100143,0.660679,0.0509849,0.796407,N/A
4,ABBV,113.81,N/A,0.241434,0.187625,0.121307,0.347305,0.0824901,0.606786,-0.0154749,0.363273,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.09,N/A,0.423069,0.409182,0.10152,0.291417,0.0915425,0.642715,-0.0228587,0.297405,N/A
501,ZBH,170.32,N/A,0.46169,0.457086,0.116641,0.327345,0.0304813,0.379242,0.0116628,0.590818,N/A
502,ZBRA,520.12,N/A,1.11396,0.904192,0.371481,0.828343,0.0724165,0.552894,0.0194799,0.628743,N/A
503,ZION,58.20,N/A,0.807378,0.762475,0.305002,0.732535,-0.000860476,0.243513,-0.0224571,0.303393,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [92]:
from statistics import mean


for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

F:\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,153.58,N/A,0.759551,0.742515,0.262317,0.660679,0.180868,0.884232,0.105249,0.938124,0.806387
1,AAL,22.22,N/A,0.825478,0.780439,0.432563,0.9002,-0.032063,0.117764,-0.072599,0.0578842,0.464072
2,AAP,206.69,N/A,0.498012,0.497006,0.283045,0.704591,0.0994839,0.654691,0.0904203,0.912176,0.692116
3,AAPL,136.61,N/A,0.531402,0.532934,0.0120478,0.107784,0.100143,0.660679,0.0509849,0.796407,0.524451
4,ABBV,113.81,N/A,0.241434,0.187625,0.121307,0.347305,0.0824901,0.606786,-0.0154749,0.363273,0.376248
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.09,N/A,0.423069,0.409182,0.10152,0.291417,0.0915425,0.642715,-0.0228587,0.297405,0.41018
501,ZBH,170.32,N/A,0.46169,0.457086,0.116641,0.327345,0.0304813,0.379242,0.0116628,0.590818,0.438623
502,ZBRA,520.12,N/A,1.11396,0.904192,0.371481,0.828343,0.0724165,0.552894,0.0194799,0.628743,0.728543
503,ZION,58.20,N/A,0.807378,0.762475,0.305002,0.732535,-0.000860476,0.243513,-0.0224571,0.303393,0.510479


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [93]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)

F:\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [94]:
portfolio_input()

In [95]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

F:\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DVN,30.46,6565,1.95039,0.992016,0.970216,0.996008,0.322117,0.992016,0.137604,0.97006,0.987525
1,MRO,14.49,13802,1.54146,0.97006,1.1419,1,0.27499,0.972056,0.184962,0.986028,0.982036
2,FANG,99.55,2009,1.41726,0.958084,1.06611,0.998004,0.279566,0.974052,0.239893,0.996008,0.981537
3,NVDA,761.68,262,1.09311,0.898204,0.486763,0.932136,0.500108,1,0.220045,0.994012,0.956088
4,DXC,39.59,5051,1.61889,0.976048,0.736089,0.986028,0.405781,0.998004,0.0689945,0.862275,0.955589
5,F,15.63,12795,1.61888,0.974052,0.715363,0.984032,0.239257,0.936128,0.0928496,0.914172,0.952096
6,DFS,128.91,1551,1.66803,0.98004,0.425699,0.892216,0.28533,0.978044,0.0710835,0.876248,0.931637
7,EXR,172.70,1158,0.904036,0.828343,0.495081,0.94012,0.271056,0.97006,0.142049,0.972056,0.927645
8,LB,75.42,2651,4.22785,1,0.867206,0.99002,0.17754,0.878244,0.0642807,0.842315,0.927645
9,OXY,33.68,5938,0.874063,0.804391,0.874966,0.992016,0.193729,0.91018,0.307721,1,0.926647


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [96]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [97]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [98]:
column_formats = {
    'A' : ['Ticker', string_template],
    'B' : ['Price', dollar_template],
    'C' : ['Number of Shares to Buy', integer_template],
    'D' : ['One-Year Price Return', percent_template],
    'E' : ['One-Year Return Percentile', percent_template],
    'F' : ['Six-Month Price Return', percent_template],
    'G' : ['Six-Month Return Percentile', percent_template],
    'H' : ['Three-Month Price Return', percent_template],
    'I' : ['Three-Month Return Percentile', percent_template],
    'J' : ['One-Month Price Return', percent_template],
    'K' : ['One-Month Return Percentile', percent_template],
    'L' : ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

writer.save()

## Saving Our Excel Output

As before, saving our Excel output is very easy: